# Modelo 5

Este módulo está associado a fechar o modelo que será utilizado no Deploy. Para isto, utilizaremos os três modelos finais considerados no Módulo 4  


É importante ressaltar que foram revistos 0 e 1 aplicados a coluna 'y' para que tenha menos carga aleatória. Além disso, foi preenchido manualmente todas as informações de acordo com o título.


In [1]:
#Importando as bibliotecas
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('raw_data_with_label_rev1.xlsx')

## Tratamento dos dados

Nesta etapa vamos retirar linhas repetidas a onde um mesmo video aparece para duas palavras-chaves (ex: Machine Learning e Kaggle).

Além disso, vamos verificar os valores ausentes.

In [3]:
#Deletando as linhas duplicadas
df.drop_duplicates(inplace = True)

In [4]:
#Verificando a quantidade de nulos
df.isnull().sum()

title          0
y              0
upload_date    2
view_count     2
query          2
dtype: int64

In [5]:
#Deletando as linhas que estão com valores ausentes
df.dropna(axis = 0, subset = ['upload_date', 'view_count', 'query'], inplace = True)

In [6]:
#Convertendo a variável date para datetime
df['upload_date'] = pd.to_datetime(df['upload_date'])

## Features

Esta etapa está associada a criação de features

In [7]:
#Armazenando os dados com y preenchido
df_preenchido = df.loc[ df['y'].notnull() ]

In [8]:
#Criando um dataframe vazio para features com o mesmo índices do df_preenchido
features = pd.DataFrame(index = df_preenchido.index)

#Criando um dataframe exclusivo para y -> Variável Alvo
y = df_preenchido['y'].copy()

**Criando uma feature de data desde da publicação**

Como os modelos de Machine Learning não conseguem ler datas, devemos criar alguma forma para transformar estas datas em dados numéricos. Portanto, iremos fazer a diferença em relação a data de hoje (31/05/2021)

In [9]:
pd.to_datetime('2021/05/31')-df_preenchido['upload_date']

0        5 days
1        5 days
2        5 days
3        5 days
4        5 days
         ...   
1505   915 days
1506   938 days
1507   942 days
1508   950 days
1509   951 days
Name: upload_date, Length: 1507, dtype: timedelta64[ns]

Como podemos observar vem com a palavras "days" e no formato timedelta64. Para contornarmos isto, utilizaremos um método do numpy chamado **timedelta64** em que passaremos como argumento a diferença de um dia, já que os dados estão associados a diferença diária. Isto permite que os valores sejam convertidos para float.

In [10]:
(pd.to_datetime('2021/05/31')-df_preenchido['upload_date'])/np.timedelta64(1, 'D')

0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
        ...  
1505    915.0
1506    938.0
1507    942.0
1508    950.0
1509    951.0
Name: upload_date, Length: 1507, dtype: float64

In [11]:
#Criando a feature de diferença diária
features['tempo_desde_pub'] = (pd.to_datetime('2021/05/31')-df_preenchido['upload_date'])/np.timedelta64(1, 'D')

**Features - Views**
Criando um conjunto de features para view.

Iremos replicar o count_views e iremos calcular as views por dia, utilizando a feature tempo_desde_pub

In [12]:
#Armazenando as views
features['views'] = df_preenchido['view_count']

#Fazendo o cálculo de views por dia
features['views_por_dia'] = features['views'] / features['tempo_desde_pub'] # Serve para avaliar a capacidade do video em promover views
                                                                            # E desconsiderar o fator tempo

Como é uma séria temporal e os dados devem ser fatiados considerando a sequência temporal, a feature **tempo_desde_pub** vai trazer a mesma informação do que a variável **date**, ou seja, se fatiássemos em 01/01/2020, os valores antes desta data seriam menores e após seriam maiores, porque seguem a mesma sequência temporal.

Diante disto, vamos deletar esta coluna do dataset

In [13]:
#Deletando a variável tempo_desde_pub
features.drop(['tempo_desde_pub'], axis = 1, inplace = True)

In [14]:
#Imprimindo as 5 primeiras linhas
features.head()

,views,views_por_dia
0,7.0,1.4
1,27.0,5.4
2,1.0,0.2
3,20.0,4.0
4,2.0,0.4


## Preparação dos Dados

In [15]:
#Separando entre treino e teste
mask_train = df_preenchido['upload_date'] < "2020-04-03" #Utilizado +- a metade do período temporal
mask_val = df_preenchido['upload_date'] >= "2020-04-03"

X_train, X_val = features[mask_train], features[mask_val]
y_train, y_val = y[mask_train], y[mask_val]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((515, 2), (992, 2), (515,), (992,))

## Random Forest

In [19]:
#Importando o método
from sklearn.feature_extraction.text import TfidfVectorizer

#Criando uma variável de títulos com os dados de treino e validação
title_train = df_preenchido.loc[mask_train, 'title']
title_val = df_preenchido.loc[mask_val, 'title']

#Instanciando o objeto TfidfVectorizer
title_vec = TfidfVectorizer(min_df = 2, ngram_range = (1,1))

#Treinando e transformando os dados de treino referente ao título
title_bow_train = title_vec.fit_transform(title_train)

#Transformando os dados de validação com os parâmetros de treino
title_bow_val = title_vec.transform(title_val)

#o termo bow significa bag of words -> devido a matriz de palavras geradas

In [20]:
#Importando o método hstack
from scipy.sparse import hstack

#Aplicando o método hstack para juntar o X com as matrizes de bag of word
X_train_wtitle = hstack([X_train, title_bow_train])
X_val_wtitle = hstack([X_val, title_bow_val])

In [21]:
#Imprimindo as dimensões
X_train_wtitle.shape, X_val_wtitle.shape

((515, 433), (992, 433))

In [22]:
#Importando as bibliotecas
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

In [23]:
#Instanciando a RandomForest
#Como os dados estão desbalanceados, vamos utilizar o argumento class_weight para dar mais peso a classe desbalanceada
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf = 1, class_weight="balanced", n_jobs=6)
mdl.fit(X_train_wtitle, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [27]:
#Pegando as probabilidade
p_rf = mdl.predict_proba(X_val_wtitle)[:,1]

In [28]:
#Calculando a metrica average_precision_score
average_precision_score(y_val, p_rf)

0.581860377926327

In [29]:
#Calculando a ROC_AUC
roc_auc_score(y_val, p_rf)

0.7551062780618204

## LighGBM

In [32]:
#Importando o modelo
from lightgbm import LGBMClassifier
#----------------------------------------------Criando a Bag of Words com os parâmetros tunados-------------------------------
    
#Instanciando o objeto TfidfVectorizer
title_vec = TfidfVectorizer(min_df = 2, ngram_range = (1,1))

#Treinando e transformando os dados de treino referente ao título
title_bow_train = title_vec.fit_transform(title_train)

#Transformando os dados de validação com os parâmetros de treino
title_bow_val = title_vec.transform(title_val)
    
 #----------------------------------------------Juntando o X com as bag of words -------------------------------------------
    
#Aplicando o método hstack para juntar o X com as matrizes de bag of word
X_train_wtitle = hstack([X_train, title_bow_train])
X_val_wtitle = hstack([X_val, title_bow_val])
    
#---------------------------------------------Tunando os parâmetros e aplicando-------------------------------------------

#Instanciando o modelo
mdl_lgm = LGBMClassifier(random_state = 42, 
                          class_weight="balanced", 
                          n_jobs=6, 
                          learning_rate = 0.005607394468548006,
                          num_leaves = 2 ** 7, #Para dar um sentido hipotético do tamanho da árvore, olhar comentário do Mario
                          max_depth = 7,
                          min_child_samples = 1,
                          subsample = 0.6655993304551099,
                          colsample_bytree = 0.9586294865021325,
                          n_estimators = 523,
                          bagging_freq = 1)

#Treinando o modelo
mdl_lgm.fit(X_train_wtitle, y_train)

#Pegando as probabilidade
p_lgm = mdl_lgm.predict_proba(X_val_wtitle)[:,1]

#Calculando a ROC_AUC
print("ROC_AUC = {}".format(roc_auc_score(y_val, p_lgm)))
    
#Retorno da função custo 
print("AP = {}".format(average_precision_score(y_val, p_lgm)))

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ROC_AUC = 0.7966338792141392
AP = 0.6402330807008632


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


#### Regressão Logística

Como a diferença entre StandarScaler e MaxAbsScaler é muito pequeno. Seria melhor optar pelo MaxAbsScaler(), já que não é necessário filtrar as variáveis categórias, já que ele divide pelo maior número da coluna e qualquer divisão com 0 será 0.

In [35]:
#Importando as bibliotecas
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [41]:
#Instanciando o modelo
lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(n_jobs = 6, random_state = 42, class_weight="balanced", C = 0.9638851701068524, solver = 'saga', penalty = 'l2'))
    
#Treinando o modelo
lr_pipeline.fit(X_train_wtitle, y_train)
    
#Prevendo o modelo
p_lr = lr_pipeline.predict_proba(X_val_wtitle)[:, 1]
    
#Calculando os scores
print("AP = {}".format(average_precision_score(y_val, p_lr) ))
print("ROC_AUC = {}".format(roc_auc_score(y_val, p_lr) ))


AP = 0.5877241385790812
ROC_AUC = 0.7701071864213231


## Ensemble

Nesta etapa, vamos combinar os modelos para verificar se melhoram os scores das métricas escolhidas.

Como os dados são relativamente simples e o conjunto é pequeno, não será aplicado modelos mais complexos de ensemble.

Lembrando dos seguintes resultados:
* Random Forest: Ap = 0.581860377926327 e ROC_AUC = 0.7551062780618204
* LighGBM: Ap = 0.6402330807008632 e ROC_AUC = 0.7966338792141392
* Regressão Logística: Ap = 0.5877241385790812 e ROC_AUC = 0.7701071864213231

O próximo passo é aplicarmos uma média simples entre os resultados dos modelos para verificar sua performance.

In [45]:
#Média Simples
p_ensemble_média_simples = (p_lgm + p_lr + p_rf) / 3

#Apurando os resultados
print("ROC_AUC = {}".format(roc_auc_score(y_val, p_ensemble_média_simples)))
print("AP = {}".format(average_precision_score(y_val, p_ensemble_média_simples)))

ROC_AUC = 0.7969738651994498
AP = 0.6299727951771387


O próximo passo é avaliarmos a correlação entre as previsões com o intuito de buscar um ensemble que tenham modelos que não estejam correlacionados para maximizar a capacidade do ensemble de "olhar pontos de vistas diferentes".

In [46]:
#Verificando a correlação
pd.DataFrame({"LR": p_lr, "RF": p_rf, "LGBM": p_lgm}).corr()

,LR,RF,LGBM
LR,1.000000,0.749385,0.746689
RF,0.749385,1.000000,0.907141
LGBM,0.746689,0.907141,1.000000


Neste caso, é complicado, os modelos estão muitos correlacionados. Vamos criar uma média ponderada com base na Regressão Logística e LGBM.

In [51]:
#Média ponderada
p_ensemble_média_ponderada = 0.6*p_lr + 0.4*p_lgm

#Apurando os resultados
print("ROC_AUC = {}".format(roc_auc_score(y_val, p_ensemble_média_ponderada)))
print("AP = {}".format(average_precision_score(y_val, p_ensemble_média_ponderada)))

ROC_AUC = 0.8005657782045625
AP = 0.6339771056912653


Anotações em relação as simulações:
* Ap = 0.6402783961532013 e ROC_AUC = 0.8043756974903326 - 0.5 LR / 0.5 LGBM
* Ap = 0.6454420943817779 e ROC_AUC = 0.8065298071682542 - 0.4 LR / 0.6 LGBM
* Ap = 0.6464997305656753 e ROC_AUC = 0.8061924164958085 - 0.3 LR / 0.7 LGBM
* Ap = 0.6457905886080421 e ROC_AUC = 0.8046767537826686 - 0.2 LR / 0.8 LGBM

Vamos utilizar o seguinte ensemble:
* Ap = 0.6464997305656753 e ROC_AUC = 0.8061924164958085 - 0.3 LR / 0.7 LGBM

## Exportando os Modelos

Nesta etapa, vamos exportar os modelos. É recomemdado fazer a exportação pelo joblib.

In [52]:
import joblib as jb

jb.dump(mdl_lgm, "lgbm.pkl.z")
jb.dump(lr_pipeline, "logistic_reg.pkl.z")
jb.dump(title_vec, "title_vectorizer.pkl.z")

['title_vectorizer.pkl.z']

Para carregar o modelo basta utilizar a linha de comando jb.load()

## Informações Importantes

* Não foi utilizado para este projeto dados de teste, apenas validação e treino. Portanto, a forma que foi utilizada pode ter overfitado um pouco os resultados, mas sem gerar problemas drásticos na predição. Isso é possível avaliar pelos scores, o resultado não deu algo muito próximo dos valores máximos, que é um grande indicativo de overfitting. Além disso, o Mario comentou que antigamente ele dividia entre treino, validação e teste, porém, com a experiência ele avaliou que não existe teste melhor do que colocar o modelo em produção para avaliar os resultados obtidos (Teste em Produção). 
* Comentário do Mario Filho sobre não retreinar com todos os dados: "Eu fazia muito isso de retreinar usando todos os dados, até encontrar um caso em que o modelo ficou bem diferente (os dados eram muito ruidosos). Em geral não acho que valha a pena retreinar fora de competições. Você acaba perdendo a sua estimativa da distribuição de probabilidades que ele prevê (e muda o ponto de corte, se tiver um). Uma alternativa a isso é fazer uma CV, tunar e salvar um modelo por fold e usar a média deles como previsão. Ainda assim acho que, na prática, a diferença no score não compensa."
* Segundo comentário do Mario Filho sobre levar para produção um modelo com pouca performance: "Boa noite, Matheus. A decisão de colocar em produção leva em conta duas coisas: se bate a solução atual e se o esforço de engenharia compensa. 20% pode ser muito ou pode ser pouco. Um AUC de 0.52 para um fundo de investimentos é muito bom. Para um sistema de recomendação em e-commerce, não é. As métricas de ML (como AP) servem só para ter uma ideia se a solução é melhor que uma baseline (normalmente a solução atual que a empresa usa), por isso eu recomendo iterar rápido e colocar em produção para ver o valor real. Tudo depende de quanto a empresa vai ganhar a mais ou gastar a menos por usar a solução. Já vi casos em que a melhora nas métricas de ML foi pequena, testes em produção mostraram que a solução nova era ligeiramente melhor e a empresa estava super feliz com os resultados de negócios. Resumindo, as métricas de ML são mais úteis como um "primeiro filtro" para tirar soluções muito ruins. Elas ajudam a gente a ter uma noção do que deve funcionar na prática (em métricas de negócios que são difíceis de medir offline), mas é comum ouvir casos em que métricas de ML estavam boas e o modelo em produção ficou ruim e vice-versa. Resumindo 2 (eu vou lembrando e comentando hehehe): pense sempre nessas métricas (qualquer métrica offline) como correlacionadas com as métricas em produção, mas não como indicadores perfeitos. "